In [2]:
#install everything

!pip install xgboost scikit-learn plotly pandas numpy joblib gradio

# ================================
# ENHANCED CHURN PREDICTION SYSTEM
# Google Colab - All-in-One
# ================================

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.impute import SimpleImputer
import xgboost as xgb
import joblib
import gradio as gr

# ----------------------------
# 1. Load & Clean Data
# ----------------------------
url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
df = pd.read_csv(url)
df = df.drop(columns=['customerID'], errors='ignore')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)

# ----------------------------
# 2. Feature Setup
# ----------------------------
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
categorical_features = ['Contract', 'PaymentMethod', 'InternetService', 'gender', 'SeniorCitizen', 'Partner']
X = df[numeric_features + categorical_features]
y = df['Churn']

# ----------------------------
# 3. Preprocessor
# ----------------------------
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# ----------------------------
# 4. Train/Test Split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ----------------------------
# 5. Hyperparameter Tuning (XGBoost)
# ----------------------------
print("🔍 Starting hyperparameter tuning (may take 1-2 mins)...")
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
])

param_dist = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.05, 0.1, 0.2]
}

xgb_random = RandomizedSearchCV(
    xgb_pipeline, param_dist, n_iter=8, cv=3,
    scoring='roc_auc', random_state=42, n_jobs=-1
)
xgb_random.fit(X_train, y_train)

best_model = xgb_random.best_estimator_
best_proba = best_model.predict_proba(X_test)[:, 1]
best_pred = best_model.predict(X_test)

print(f"✅ Best AUC: {xgb_random.best_score_:.4f}")
print(f"✅ Best params: {xgb_random.best_params_}")

# ----------------------------
# 6. Evaluation
# ----------------------------
acc = accuracy_score(y_test, best_pred)
auc = roc_auc_score(y_test, best_proba)
print(f"\n📊 Final Test Results:\n   Accuracy: {acc:.2%}\n   AUC:      {auc:.2%}")

# ----------------------------
# 7. Feature Importance
# ----------------------------
preprocessor.fit(X_train)
cat_features = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
feature_names = numeric_features + list(cat_features)
importance = best_model.named_steps['classifier'].feature_importances_
feat_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance}).sort_values('Importance', ascending=False)

fig1 = px.bar(feat_imp_df.head(10), x='Importance', y='Feature', orientation='h',
              title='Top 10 Churn Drivers', color='Importance', color_continuous_scale='Blues')
fig1.update_layout(yaxis={'categoryorder':'total ascending'})
fig1.show()

# ----------------------------
# 8. Churn Probability Distribution
# ----------------------------
fig2 = go.Figure()
fig2.add_trace(go.Histogram(x=best_proba, nbinsx=50, opacity=0.75))
fig2.add_vline(x=0.5, line_dash="dash", line_color="red", annotation_text="Default Threshold")
fig2.update_layout(title="Churn Probability Distribution", xaxis_title="Probability", yaxis_title="Count")
fig2.show()

# ----------------------------
# 9. Precision-Recall vs Threshold
# ----------------------------
precision, recall, thresholds = precision_recall_curve(y_test, best_proba)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=thresholds, y=recall[:-1], mode='lines', name='Recall'))
fig3.add_trace(go.Scatter(x=thresholds, y=precision[:-1], mode='lines', name='Precision'))
fig3.update_layout(title="Precision & Recall vs Threshold", xaxis_title="Threshold", yaxis_title="Score")
fig3.show()

# Choose optimal threshold (max F1 not shown, but 0.4 often better than 0.5)
optimal_threshold = 0.4  # You can adjust based on business needs
print(f"\n💡 Using optimal threshold: {optimal_threshold:.1f} (instead of 0.5)")

# ----------------------------
# 10. High-Risk Customers + Retention Strategy
# ----------------------------
X_test_full = X_test.copy()
X_test_full['Churn_Prob'] = best_proba
X_test_full['High_Risk'] = (best_proba >= optimal_threshold).astype(int)
X_test_full['Actual_Churn'] = y_test.values

# Add retention strategy
def assign_strategy(row):
    if row['Contract'] == 'Month-to-month':
        if row['InternetService'] == 'Fiber optic':
            return "Offer 1-year contract + $10 discount"
        else:
            return "Offer 1-year contract"
    elif row['SeniorCitizen'] == '1':
        return "Call + free tech support"
    elif row['tenure'] < 12 and row['MonthlyCharges'] > 70:
        return "Loyalty cashback offer"
    else:
        return "General retention offer"

X_test_full['Retention_Strategy'] = X_test_full.apply(assign_strategy, axis=1)

# Show top 10 high-risk customers
high_risk_customers = X_test_full[X_test_full['High_Risk'] == 1].sort_values('Churn_Prob', ascending=False)
print("\n📋 Top 10 High-Risk Customers (with Retention Strategy):")
print(high_risk_customers[['Churn_Prob', 'Contract', 'tenure', 'MonthlyCharges', 'Retention_Strategy']].head(10))

# Save high-risk list
high_risk_customers.to_csv("high_risk_customers.csv", index=False)
print("\n💾 High-risk customer list saved: 'high_risk_customers.csv'")

# ----------------------------
# 11. Save Model
# ----------------------------
joblib.dump(best_model, 'churn_model.pkl')
print("💾 Model saved as 'churn_model.pkl'")

# ----------------------------
# 12. Gradio Web App (Interactive Demo)
# ----------------------------
def predict_churn(tenure, MonthlyCharges, TotalCharges, Contract, InternetService, gender, SeniorCitizen, Partner):
    input_df = pd.DataFrame([{
        'tenure': tenure,
        'MonthlyCharges': MonthlyCharges,
        'TotalCharges': TotalCharges,
        'Contract': Contract,
        'PaymentMethod': 'Electronic check',  # placeholder (not used in strategy)
        'InternetService': InternetService,
        'gender': gender,
        'SeniorCitizen': str(int(SeniorCitizen)),
        'Partner': Partner
    }])
    prob = best_model.predict_proba(input_df)[0][1]
    risk = "High Risk 🚨" if prob >= optimal_threshold else "Low Risk ✅"

    # Apply strategy
    strategy = "General retention offer"
    if Contract == 'Month-to-month':
        if InternetService == 'Fiber optic':
            strategy = "Offer 1-year contract + $10 discount"
        else:
            strategy = "Offer 1-year contract"
    elif SeniorCitizen:
        strategy = "Call + free tech support"
    elif tenure < 12 and MonthlyCharges > 70:
        strategy = "Loyalty cashback offer"

    return {
        "Churn Probability": f"{prob:.2%}",
        "Risk Level": risk,
        "Recommended Action": strategy
    }

# Launch Gradio
demo = gr.Interface(
    fn=predict_churn,
    inputs=[
        gr.Number(label="Tenure (months)", value=10),
        gr.Number(label="Monthly Charges ($)", value=70),
        gr.Number(label="Total Charges ($)", value=700),
        gr.Dropdown(["Month-to-month", "One year", "Two year"], label="Contract", value="Month-to-month"),
        gr.Dropdown(["DSL", "Fiber optic", "No"], label="Internet Service", value="Fiber optic"),
        gr.Radio(["Female", "Male"], label="Gender", value="Male"),
        gr.Checkbox(label="Senior Citizen", value=False),
        gr.Radio(["Yes", "No"], label="Partner", value="No")
    ],
    outputs="json",
    title="🎯 Customer Churn Predictor",
    description="Enter customer details to predict churn risk and get retention recommendations."
)

print("\n🚀 Launching Gradio app... (click the public link when ready)")
demo.launch(share=True)

/tmp/ipython-input-134525401.py:32: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





🔍 Starting hyperparameter tuning (may take 1-2 mins)...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning:

[21:20:01] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.




✅ Best AUC: 0.8435
✅ Best params: {'classifier__n_estimators': 100, 'classifier__max_depth': 3, 'classifier__learning_rate': 0.05}

📊 Final Test Results:
   Accuracy: 79.84%
   AUC:      84.18%



💡 Using optimal threshold: 0.4 (instead of 0.5)

📋 Top 10 High-Risk Customers (with Retention Strategy):
      Churn_Prob        Contract  tenure  MonthlyCharges  \
2246    0.860984  Month-to-month       1          102.45   
3380    0.855422  Month-to-month       1           95.10   
4585    0.846509  Month-to-month       1           85.05   
6748    0.846509  Month-to-month       1           85.00   
1731    0.831314  Month-to-month       1           69.60   
2900    0.831314  Month-to-month       1           69.25   
2888    0.826091  Month-to-month       1           71.00   
6623    0.826091  Month-to-month       1           76.45   
2036    0.826091  Month-to-month       1           74.30   
642     0.824564  Month-to-month       1           89.55   

                        Retention_Strategy  
2246  Offer 1-year contract + $10 discount  
3380  Offer 1-year contract + $10 discount  
4585  Offer 1-year contract + $10 discount  
6748  Offer 1-year contract + $10 discount  
1731  Of